- 각 그룹을 마다 target의 시계열 모양에 따라 다르게 생각?
    - trade data를 통해 그룹 clustering해볼까

- target의 값을 quantile로 그룹핑하여 각각 모델링?
    - 해당 모델을 통해 predict average?
    - 근데 그러면 큰 target을 학습한 모델의 의견이 거의 따라질듯?
    - 그러면 가중치를 좀 다르게 해서?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

trade = pd.read_csv('trade_train.csv', index_col=0)
stock = pd.read_csv('stocks.csv', index_col=0)
answer = pd.read_csv('answer_sheet.csv')

In [2]:
from DataSet import makeDataset, makeCV, encoding, makeSub
from myModel import modelCatboost, modelLightgbm, linear

In [3]:
seq2seq_df = pd.read_csv('seq2seq_df.csv')

# val과 test X feature 비교 by modeling (adverserial)

In [10]:
# df_roling2_diff1 = makeDataset(seq2seq_df, rolling_range=[2], diff_range=[1])

# X_train_cat, y_train_cat, X_val_cat, y_val_cat, X_test_cat = \
#     makeCV(df_roling2_diff1, train_=[201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004], use_catboost=True)

# adv_val = X_val_cat.copy()
# adv_val['target'] = 1
# adv_test = X_test_cat.copy()
# adv_test['target'] = 0

# adv_train = pd.concat([adv_val, adv_test], axis=0).reset_index(drop=True)

# from catboost import CatBoostClassifier

# category_cols = ['종목번호', '시장구분', '표준산업구분코드_대분류','그룹번호']

# params = {
#     'iterations': 2000,
#     'learning_rate': 0.05,
#     'random_seed': 42,
#     'task_type' : 'GPU',
#     'early_stopping_rounds' : 500,
#     'eval_metric' : 'AUC'
# }

# train_pool = Pool(adv_train.iloc[:, :-2], 
#                   adv_train.iloc[:, -1], 
#                   cat_features=category_cols)

# model = CatBoostClassifier(**params)
# model.fit(train_pool, verbose=200)

# print('catboost feature importance')
# feature_importances = model.get_feature_importance(train_pool)
# feature_names = adv_train.iloc[:, :-2].columns
# for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
#     print('{}: {}'.format(name, score))

cut use : False, quantile : 0.99, rolling range: [2], diff range : [1]


# Df 만들기

In [4]:
df_rolling23_diff12_NoMinMax = makeDataset(seq2seq_df, rolling_range=[2,3], diff_range=[1,2], save_na=False, use_minmax=False)

# df_rolling23_diff1_NoMinMax = makeDataset(seq2seq_df, rolling_range=[2,3], diff_range=[1], save_na=False, use_minmax=False)

# df_rolling2_diff12_NoMinmax = makeDataset(seq2seq_df, rolling_range=[2], diff_range=[1,2], save_na=False, use_minmax=False)

df_rolling2_diff1_NoMinmax = makeDataset(seq2seq_df, rolling_range=[2], diff_range=[1], save_na=False, use_minmax=False)

cut use : False, quantile : 0.99, rolling range: [2, 3], diff range : [1, 2], use MinMAx : False
cut use : False, quantile : 0.99, rolling range: [2], diff range : [1], use MinMAx : False


In [5]:
df_rolling23_diff12_NoMinMax.head()

,종목번호,min,max,mean,std,기준년월,거래금액_만원단위,거래량1,거래량2,거래량3,...,매수고객수rolling_std3,매도고객수diff1,매도고객수diff2,매수고객수diff1,매수고객수diff2,group_char1,group_char2,group_char3,group_char4,target
0,A000100,217500,233000,226789.47,4394.81,201909,894332.84,24135.4,72702.0,52907.6,...,0.00,0.0,0.0,0.0,0.0,0,1,0,0,0.0
1,A000100,217500,233000,226789.47,4394.81,201909,894332.84,24135.4,72702.0,52907.6,...,0.00,0.0,0.0,0.0,0.0,0,0,0,1,0.0
2,A000100,217500,233000,226789.47,4394.81,201909,894332.84,24135.4,72702.0,52907.6,...,0.00,0.0,0.0,0.0,0.0,0,0,1,0,0.0
3,A000100,217500,233000,226789.47,4394.81,201909,894332.84,24135.4,72702.0,52907.6,...,4.62,0.0,-4.0,0.0,-8.0,1,0,0,0,0.0
4,A000100,217500,233000,226789.47,4394.81,201909,894332.84,24135.4,72702.0,52907.6,...,2.08,-3.0,-3.0,-3.0,-4.0,1,0,0,0,0.0


# catboost

1. SaveNA : not good
2. FS : only about time, 매수고객수만 해도 비슷한 성능 (매도고객수는 일단 drop)
3. big parameter change : grow_policy(Lossguide, Depthwise가 val rmse는 너 낮게 하는데 lb는 별로), metric(차이없음 그냥 rmse 고) 
    - SymmetricTree : lb가 57점으로 가장 잘 나옴\
    
# 4. ensemble
## - company model 1 + time model 2 voting (company+매도는 cat, 매수는 lgb로 해볼까)
## - 6월에 가장 팔린애들 2개 + catboost 예측 1개 voting
# 5. 5월도 val하지 않고 fitting

In [11]:
X_train_cat, y_train_cat, X_val_cat, y_val_cat, X_test_cat = \
    makeCV(df_rolling23_diff12_NoMinMax, train_=[201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004], use_catboost=True)

X_train, y_train, X_val, y_val, X_test = \
    makeCV(df_rolling23_diff12_NoMinMax, train_=[201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004], use_catboost=False)

weight_idx : [0, 6144, 12336, 18528, 24864, 31248, 37680, 44112]
weight_idx : [0, 6144, 12336, 18528, 24864, 31248, 37680, 44112]


In [12]:
X_train_cat.columns

Index(['종목번호', 'min', 'max', 'mean', 'std', '거래금액_만원단위', '거래량1', '거래량2',
       '거래량3', '거래량4', '시장구분', '표준산업구분코드_대분류', '그룹번호', '그룹내고객수', '매수고객수',
       '매도고객수', '매도고객수rolling_mean2', '매도고객수rolling_mean3',
       '매도고객수rolling_std2', '매도고객수rolling_std3', '매수고객수rolling_mean2',
       '매수고객수rolling_mean3', '매수고객수rolling_std2', '매수고객수rolling_std3',
       '매도고객수diff1', '매도고객수diff2', '매수고객수diff1', '매수고객수diff2', 'group_char1',
       'group_char2', 'group_char3', 'group_char4', 'year'],
      dtype='object')

## cat

In [35]:
# catboost
params = {
    'iterations': 5000,
    'learning_rate': 0.03,
    'random_seed': 42,
    'use_best_model': True,
    'task_type' : 'GPU',
    'early_stopping_rounds' : 700,
    'eval_metric' : 'RMSE' 
}
category_cols = ['종목번호', '표준산업구분코드_대분류','그룹번호']
#weight = {0:1, 6192:1, 12528:1, 18912:1, 25344:2, 31776:2, 38208:3, 44640:3}
#weight = {0:1, 6144:1, 12336:1, 18528:2, 24864:2, 31248:2, 37680:3}
weight = {0:1, 6144:1, 12336:1, 18528:1, 24864:3, 31248:3, 37680:3, 44112:3}
selected_feature = ['종목번호', '거래금액_만원단위', '거래량1', '거래량2',
       '거래량3', '거래량4', '표준산업구분코드_대분류', '그룹번호', '그룹내고객수', '매수고객수',
       '매도고객수', '매도고객수rolling_mean2', '매도고객수rolling_mean3',
       '매도고객수rolling_std2', '매도고객수rolling_std3', '매수고객수rolling_mean2',
       '매수고객수rolling_mean3', '매수고객수rolling_std2', '매수고객수rolling_std3',
       '매도고객수diff1', '매도고객수diff2', '매수고객수diff1', '매수고객수diff2', 'group_char1','group_char4', 'year']

In [36]:
model_cat, pred_train_cat, pred_val_cat, pred_test_cat=\
    modelCatboost(X_train_cat, y_train_cat, X_val_cat, y_val_cat, X_test_cat , \
                  category_cols, weight=weight, params=params, selected_feature=selected_feature)

0:	learn: 13.1695907	test: 20.8556808	best: 20.8556808 (0)	total: 20ms	remaining: 1m 40s
100:	learn: 7.7315810	test: 13.9991671	best: 13.9991671 (100)	total: 1.75s	remaining: 1m 25s
200:	learn: 6.8735059	test: 13.0023692	best: 13.0021450 (199)	total: 3.74s	remaining: 1m 29s
300:	learn: 6.6841153	test: 12.7213051	best: 12.7213051 (300)	total: 5.72s	remaining: 1m 29s
400:	learn: 6.4976784	test: 12.4763415	best: 12.4703029 (398)	total: 7.69s	remaining: 1m 28s
500:	learn: 6.4021154	test: 12.3403986	best: 12.3399616 (498)	total: 9.57s	remaining: 1m 25s
600:	learn: 6.3322206	test: 12.2078193	best: 12.2078193 (600)	total: 11.5s	remaining: 1m 24s
700:	learn: 6.3010706	test: 12.1324756	best: 12.1324416 (699)	total: 13.4s	remaining: 1m 22s
800:	learn: 6.2856662	test: 12.0791028	best: 12.0783611 (799)	total: 15.3s	remaining: 1m 20s
900:	learn: 6.2560647	test: 12.0089647	best: 12.0074033 (899)	total: 17.3s	remaining: 1m 18s
1000:	learn: 6.2303516	test: 11.9675474	best: 11.9675474 (1000)	total: 19.

In [37]:
feature_importances = model_cat.get_feature_importance()
feature_names = X_train_cat[selected_feature].columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

매수고객수: 12.093610310079113
매수고객수rolling_mean3: 11.532587636104399
매수고객수diff2: 8.678626721627106
매도고객수rolling_mean2: 7.946725945690698
매도고객수: 7.827160652530905
매수고객수rolling_mean2: 7.098260310489136
그룹번호: 5.894749736825388
종목번호: 5.752847861854087
매도고객수rolling_std3: 5.73258100877845
매수고객수rolling_std2: 4.471144787455622
매도고객수diff1: 4.001219925259454
매도고객수rolling_mean3: 3.6724673511822012
매도고객수diff2: 3.489779045705653
매수고객수rolling_std3: 2.8451833729900904
매도고객수rolling_std2: 2.722942137241994
매수고객수diff1: 2.6736425280208667
거래량3: 0.8752338802937513
표준산업구분코드_대분류: 0.6592435714723118
거래량1: 0.6499811376406484
거래량4: 0.5541806463972407
거래량2: 0.3892712270901864
그룹내고객수: 0.18581004118368033
거래금액_만원단위: 0.15201089477817598
year: 0.09198768033038023
group_char1: 0.008751588978296145
group_char4: 0.0


In [31]:
cat_answer = makeSub(X_test_cat, pred_test_cat)

In [32]:
cat_answer.to_csv('0913_cat_fs_roll23_diff12_groupchar.csv', index=False)

## lgb
- encoding할때 최근 값만 이용?

In [30]:
category_cols = ['종목번호','그룹번호']
X_train, X_val, X_test = encoding(X_train, y_train, X_val, X_test, category_cols=category_cols)

In [31]:
X_train.columns

Index(['종목번호', 'min', 'max', 'mean', 'std', '거래금액_만원단위', '거래량1', '거래량2',
       '거래량3', '거래량4', '시장구분', '표준산업구분코드_대분류', '그룹번호', '그룹내고객수', '매수고객수',
       '매도고객수', '매도고객수rolling_mean2', '매도고객수rolling_mean3',
       '매도고객수rolling_std2', '매도고객수rolling_std3', '매수고객수rolling_mean2',
       '매수고객수rolling_mean3', '매수고객수rolling_std2', '매수고객수rolling_std3',
       '매도고객수diff1', '매도고객수diff2', '매수고객수diff1', '매수고객수diff2', 'year'],
      dtype='object')

In [32]:
# lgb
params = {'objective': 'regression',
             'metric': 'rmse',
             'boosting_type': 'gbdt',
             'learning_rate': 0.01,
             'seed': 42,
             'num_iterations' : 5000,
             'early_stopping_rounds' : 500
            }
#weight = {0:1, 6192:1, 12528:1, 18912:1, 25344:2, 31776:2, 38208:3, 44640:3}
weight = {0:1, 6144:1, 12336:1, 18528:2, 24864:2, 31248:2, 37680:3}
selected_feature = ['종목번호','그룹번호', '매수고객수',
       '매수고객수rolling_mean2','매수고객수rolling_mean3', '매수고객수rolling_std2', '매수고객수rolling_std3',
       '매수고객수diff1', '매수고객수diff2']

In [33]:
model, pred_train, pred_val, pred_test=\
    modelLightgbm(X_train, y_train, X_val, y_val, X_test , \
                  category_cols, weight=weight, params=params, selected_feature=selected_feature)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001023 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 905
[LightGBM] [Info] Number of data points in the train set: 44400, number of used features: 9
[LightGBM] [Info] Start training from score 1.909432
Training until validation scores don't improve for 500 rounds
[100]	training's rmse: 7.16952	valid_1's rmse: 11.2148
[200]	training's rmse: 5.46712	valid_1's rmse: 8.38786
[300]	training's rmse: 4.93311	valid_1's rmse: 7.73238
[400]	training's rmse: 4.66864	valid_1's rmse: 7.56708
[500]	training's rmse: 4.47889	valid_1's rmse: 7.52825
[600]	training's rmse: 4.33632	valid_1's rmse: 7.55366
[700]	training's rmse: 4.22094	valid_1's rmse: 7.55192
[800]	training's rmse: 4.12954	valid_1's rmse: 7.57242
[900]	training's rmse: 4.036	valid_1's rmse: 7.63942
Early stopping, best iteration is:
[47

In [34]:
feature_importances = model.feature_importance()
feature_names = X_train[selected_feature].columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

매수고객수rolling_mean3: 2368
매수고객수rolling_std3: 2004
매수고객수: 1767
매수고객수rolling_mean2: 1681
종목번호: 1589
매수고객수diff1: 1317
매수고객수diff2: 1188
매수고객수rolling_std2: 1167
그룹번호: 1019


# ensemble

일단 target pred 더해서 결과물

In [35]:
cat_lgb_sum = pred_test_cat + pred_test

In [36]:
cat_answer = makeSub(X_test_cat, cat_lgb_sum)

In [37]:
cat_answer.to_csv('0913_catAndLgb_fsDivide_roll23_diff12_dropNa_NoMinMax.csv', index=False)

# voting
- 그룹마다 특징 이전보다 많이 사는~! 경우의 수를 count
- 6월달 가장 잘 팔린 2개 + catboost 가장 잘 팔린 1개
- lgb encoding : 2020년도 target만 해보기!

In [97]:
# 6월
month6 = seq2seq_df.loc[seq2seq_df['기준년월']==202006, ['종목번호','그룹번호','매수고객수']]

onlyMonth6 = pd.read_csv('answer_sheet.csv')
result_cols = ['종목번호','그룹번호']
sub = X_test_cat[result_cols].reset_index(drop=True)

tmp = pd.merge(X_test_cat, month6, on=['종목번호','그룹번호'], how='left')

sub['pred'] = tmp['매수고객수_y']
sub = sub.sort_values(by=['그룹번호','pred'], ascending=[True, False])
group_num = sub['그룹번호'].unique()

sub_cat = X_test_cat[result_cols].reset_index(drop=True)
sub_cat['pred'] = pred_test_cat
sub_cat = sub_cat.sort_values(by=['그룹번호','pred'], ascending=[True, False])
# sub_cat = sub_cat.sort_values(by=['그룹번호','pred'], ascending=[True, True]) # 오히려 cat이 그나마 rare한 경우 맞추도록

for num in group_num:
    val_cat = sub_cat.loc[sub_cat['그룹번호']==num][:2]['종목번호'].sort_values().values
    val = sub.loc[sub['그룹번호']==num][:3]['종목번호'].sort_values().values
    onlyMonth6.loc[onlyMonth6['그룹명']==num,'종목번호1':'종목번호2'] = val_cat
    for value in val:
        if value not in val:
            onlyMonth6.loc[onlyMonth6['그룹명']==num,'종목번호3'] = value
            break

# 최종 sort
for num in group_num:
     onlyMonth6.loc[onlyMonth6['그룹명']==num,'종목번호1':] = \
        pd.Series(onlyMonth6.loc[onlyMonth6['그룹명']==num,'종목번호1':].values[0]).sort_values().values

In [98]:
onlyMonth6.to_csv('0913_onlyMonth6big1_catbig2_voting.csv', index=False)

# Analysis

- 그룹마다 특징 이전보다 많이 사는~! 경우의 수를 count
- 거래랑은 okay 나머지는 별로인듯
    - 나중에 추가적으로 feauture 생성방법 생각해보기